In [1]:
import pandas as pd
from pydruid.db import connect
import json
from time import time

In [2]:
with open('test_result.json') as json_file:
    test_result = json.load(json_file)

In [4]:
con = connect(host='localhost', port=8890, path='/druid/v2/sql/', scheme='http')

In [6]:
test_result["timings"]["druid"] = {}

for label, query in test_result["queries"].items():
    print(f"Executing {label} -> {query}")
    test_result["timings"]["druid"][label] = []

    for i in range(0, 10):
        try:
            start = time()
            pd.read_sql_query(query.replace("InvoiceDate", "__time"), con=con)
            end = time()

            total = end-start
            test_result["timings"]["druid"][label].append(total)
        except:
            test_result["timings"]["druid"][label].append(0)
            print(f"Query {query} not executed.")
            break

with open('test_result.json', 'w') as outfile:
    json.dump(test_result, outfile)

Executing Q1 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q2 -> SELECT Country, SUM(Price*Quantity) as country_sum FROM online_retail GROUP BY Country ORDER BY country_sum DESC
Executing Q3 -> SELECT StockCode, COUNT(*) as n_items FROM online_retail GROUP BY StockCode
Executing Q4 -> SELECT CustomerID, SUM(Price*Quantity) as customer_sum FROM online_retail GROUP BY CustomerID ORDER BY customer_sum DESC
Executing Q5 -> SELECT InvoiceDate, SUM(Price*Quantity) as price_sum FROM online_retail GROUP BY InvoiceDate  ORDER BY price_sum
Executing Q6 -> SELECT CustomerID, Price*Quantity as total_price FROM online_retail WHERE Quantity > 20 ORDER BY total_price DESC
Query SELECT CustomerID, Price*Quantity as total_price FROM online_retail WHERE Quantity > 20 ORDER BY total_price DESC not executed.
Executing Q7 -> SELECT InvoiceDate, AVG(Price*Quantity) as avg_total FROM online_retail GROUP BY InvoiceDate
